In [5]:
# ============================================
# MODULE 1: Topic Input & Paper Search (FINAL)
# ============================================

!pip install semanticscholar python-dotenv requests tqdm -q

import json
import os
import logging
from tqdm import tqdm
from datetime import datetime
from semanticscholar import SemanticScholar
from dotenv import load_dotenv

# --------------------------------------------
# 1. LOGGING SETUP
# --------------------------------------------

os.makedirs("logs", exist_ok=True)

logging.basicConfig(
    filename="logs/module1.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logging.info("Module 1 started.")


# --------------------------------------------
# 2. SETUP API KEY (YOU WILL ADD YOUR KEY)
# --------------------------------------------
def setup_api_key():
    """Set up API key either from .env file or directly"""
    # Method 1: Try loading from .env file
    load_dotenv()
    API_KEY = os.getenv("SEMANTIC_SCHOLAR_API_KEY")

    # Method 2: If not in .env, use direct key
    if not API_KEY:
        # Create .env file with your API key
        with open(".env", "w") as f:
            f.write("SEMANTIC_SCHOLAR_API_KEY=qD0NJlxPKa8FlwwWm16yD4KZAybbTQQ72ugbyIA3\n")
        load_dotenv()
        API_KEY = os.getenv("SEMANTIC_SCHOLAR_API_KEY")

    # Initialize Semantic Scholar
    if API_KEY:
        sch = SemanticScholar(api_key=API_KEY)
        print("Semantic Scholar initialized with API key")
    else:
        sch = SemanticScholar()
        print(" Using Semantic Scholar without API key (limited rate)")

    return sch


# --------------------------------------------
# 3. SEARCH PAPERS WITH FILTERS
# --------------------------------------------
def search_papers(topic, limit=20, year_min=None, year_max=None, min_citations=None):
    print(f"\n🔍 Searching for papers on: {topic}")
    sch = setup_api_key()

    try:
        results = sch.search_paper(
            query=topic,
            limit=limit,
            fields=[
                "paperId", "title", "abstract", "year", "authors",
                "citationCount", "openAccessPdf", "url", "venue"
            ]
        )

        papers = []
        for paper in tqdm(results, desc="Processing papers", ncols=80):

            # ------------ Filters ------------
            if paper.year and year_min and paper.year < year_min:
                continue
            if paper.year and year_max and paper.year > year_max:
                continue
            if min_citations and paper.citationCount < min_citations:
                continue

            papers.append({
                "title": paper.title,
                "authors": [a['name'] for a in paper.authors] if paper.authors else [],
                "year": paper.year,
                "paperId": paper.paperId,
                "abstract": (paper.abstract[:300] + "...") if paper.abstract else "No abstract available",
                "citationCount": paper.citationCount,
                "venue": getattr(paper, "venue", None),
                "url": paper.url,
                "pdf_url": paper.openAccessPdf['url'] if paper.openAccessPdf else None,
                "has_pdf": bool(paper.openAccessPdf)
            })

        logging.info(f"Search complete: {len(papers)} papers after filtering.")
        print("\n✔ Search complete!")

        return {
            "topic": topic,
            "timestamp": datetime.now().isoformat(),
            "filters": {
                "year_min": year_min,
                "year_max": year_max,
                "min_citations": min_citations
            },
            "total_results": len(papers),
            "papers_with_pdf": sum(1 for p in papers if p["has_pdf"]),
            "papers": papers
        }

    except Exception as e:
        print(f"❌ Error: {e}")
        logging.error(f"Search error → {e}")
        return None


# --------------------------------------------
# 4. SAVE SEARCH RESULTS
# --------------------------------------------
def save_search_results(data, filename=None):
    if not filename:
        safe_topic = "".join(c for c in data["topic"] if c.isalnum() or c == " ").replace(" ", "_")
        filename = f"{safe_topic}_search_results.json"

    os.makedirs("data/search_results", exist_ok=True)
    filepath = os.path.join("data/search_results", filename)

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    logging.info(f"Results saved: {filepath}")
    return filepath


# --------------------------------------------
# 5. DISPLAY RESULTS
# --------------------------------------------
def display_search_results(data, max_display=10):
    papers = data["papers"]

    print("\n" + "="*80)
    print(f"SEARCH RESULTS for TOPIC: {data['topic']}")
    print("="*80)

    print(f"\n📊 Statistics:")
    print(f"   • Total papers: {len(papers)}")
    print(f"   • Papers with PDF: {data['papers_with_pdf']}")

    print("\n📄 Top Papers:")
    for i, p in enumerate(papers[:max_display]):
        print(f"\n{i+1}. {p['title']}")
        print(f"   🧑 Authors: {', '.join(p['authors'][:3])}")
        print(f"   📅 Year: {p['year']} | 📈 Citations: {p['citationCount']}")
        print(f"   📄 PDF: {'✔ Available' if p['has_pdf'] else '❌ Not Available'}")
        print(f"   📝 Abstract: {p['abstract'][:150]}...\n")


# --------------------------------------------
# 6. MAIN SEARCH FUNCTION
# --------------------------------------------
def main_search():
    print("\n" + "="*80)
    print("MODULE 1: TOPIC INPUT & PAPER SEARCH")
    print("="*80)

    topic = input("\nEnter topic: ").strip() or "machine learning"

    print("\nOptional Filters:")
    year_min = input("  Minimum Year: ").strip()
    year_max = input("  Maximum Year: ").strip()
    min_citations = input("  Minimum Citations: ").strip()

    year_min = int(year_min) if year_min else None
    year_max = int(year_max) if year_max else None
    min_citations = int(min_citations) if min_citations else None

    results = search_papers(
        topic,
        limit=25,
        year_min=year_min,
        year_max=year_max,
        min_citations=min_citations
    )

    if results:
        save_path = save_search_results(results)
        display_search_results(results)
        print("\n🎉 Module 1 completed!")
        print(f"Results saved to: {save_path}")
        print("Proceed to Module 2 for PDF download.\n")
        return results, save_path

    else:
        print("❌ No results found.")
        return None, None


if __name__ == "__main__":
    main_search()



MODULE 1: TOPIC INPUT & PAPER SEARCH

Enter topic: yoga

Optional Filters:
  Minimum Year: 1990
  Maximum Year: 2025
  Minimum Citations: 1

🔍 Searching for papers on: yoga
Semantic Scholar initialized with API key


Processing papers: 1000it [03:33,  4.69it/s]


✔ Search complete!

SEARCH RESULTS for TOPIC: yoga

📊 Statistics:
   • Total papers: 974
   • Papers with PDF: 974

📄 Top Papers:

1. Reducing Stress with Yoga: A Systematic Review Based on Multimodal Biosignals
   🧑 Authors: A. Khajuria, Amit Kumar, Deepak Joshi
   📅 Year: 2024 | 📈 Citations: 24
   📄 PDF: ✔ Available
   📝 Abstract: Stress is an enormous concern in our culture because it is the root cause of many health issues. Yoga asanas and mindfulness-based practices are becom...


2. Deep Learning-Based Yoga Posture Recognition Using the Y_PN-MSSD Model for Yoga Practitioners
   🧑 Authors: Aman Upadhyay, Niha Kamal Basha, Balasundaram Ananthakrishnan
   📅 Year: 2023 | 📈 Citations: 44
   📄 PDF: ✔ Available
   📝 Abstract: In today’s digital world, and in light of the growing pandemic, many yoga instructors opt to teach online. However, even after learning or being train...


3. Yoga Meets Intelligent Internet of Things: Recent Challenges and Future Directions
   🧑 Authors: Rishi Pa

MODULE-2

In [6]:
# ============================================
# MODULE 2: Paper Selection & PDF Download (UPDATED - Option C)
# ============================================
# Features:
# - Robust PDF download with retries & exponential backoff
# - Improved verification (content-type, magic bytes, PyMuPDF load)
# - SHA-256 hash for each file
# - Organized folder layout:
#     /downloads/pdfs
#     /downloads/metadata
#     /data/dataset (papers.csv + papers.jsonl)
#     /data/reports
#     /logs
# - Progress bars via tqdm
# - Logging to logs/module2.log
# - Metadata saved per paper + combined dataset (CSV + JSONL)
# - Hooks for later summarization/embedding
# ============================================

!pip install PyMuPDF requests tqdm pandas -q

import os
import json
import time
import hashlib
import logging
import requests
import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from typing import List, Dict

# -----------------------
# SETUP FOLDERS & LOGGING
# -----------------------
os.makedirs("downloads/pdfs", exist_ok=True)
os.makedirs("downloads/metadata", exist_ok=True)
os.makedirs("data/dataset", exist_ok=True)
os.makedirs("data/reports", exist_ok=True)
os.makedirs("logs", exist_ok=True)

LOGFILE = "logs/module2.log"
logging.basicConfig(
    filename=LOGFILE,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)
logging.info("Module 2 started.")

# -----------------------
# UTIL: SAFE FILENAME
# -----------------------
def safe_filename(s, max_len=100):
    allowed = "-_.() abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    cleaned = "".join(ch for ch in s if ch in allowed).strip()
    if len(cleaned) > max_len:
        cleaned = cleaned[:max_len].rstrip()
    return cleaned or "paper"

# -----------------------
# UTIL: SHA256
# -----------------------
def sha256_of_bytes(b: bytes) -> str:
    h = hashlib.sha256()
    h.update(b)
    return h.hexdigest()

def sha256_of_file(path: str) -> str:
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

# -----------------------
# 1. LOAD SEARCH RESULTS
# -----------------------
def load_search_results(filepath: str = None):
    """
    Load the most recent saved search results if filepath not provided.
    """
    if not filepath:
        results_dir = "data/search_results"
        if not os.path.exists(results_dir):
            print("No search results directory. Run Module 1 first.")
            return None
        json_files = [f for f in os.listdir(results_dir) if f.endswith(".json")]
        if not json_files:
            print("No search results found. Run Module 1 first.")
            return None
        # newest file
        json_files.sort(key=lambda x: os.path.getmtime(os.path.join(results_dir, x)), reverse=True)
        filepath = os.path.join(results_dir, json_files[0])
        print(f"Loading most recent search results: {json_files[0]}")

    try:
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)
        print(f"Loaded {len(data.get('papers', []))} papers for topic '{data.get('topic')}'")
        return data
    except Exception as e:
        logging.exception("Failed to load search results")
        print(f"Error loading search results: {e}")
        return None

# -----------------------
# 2. FILTER / RANK / SELECT
# -----------------------
def filter_papers_with_pdfs(papers: List[Dict]) -> List[Dict]:
    out = []
    for p in papers:
        pdf_url = (p.get("pdf_url") or "").strip()
        if not pdf_url:
            continue
        # heuristic checks
        u = pdf_url.lower()
        if u.endswith(".pdf") or ".pdf?" in u or "pdf" in u:
            out.append(p)
        else:
            # still include: many OA providers use redirectors (keep them)
            out.append(p)
    logging.info(f"filter_papers_with_pdfs: {len(out)} papers with potential PDFs")
    return out

def rank_papers(papers: List[Dict]) -> List[Dict]:
    # Coerce missing counts to 0, missing year to 0 for sort stability
    def keyfn(x):
        return (x.get("citationCount") or 0, x.get("year") or 0)
    ranked = sorted(papers, key=keyfn, reverse=True)
    return ranked

def select_top_papers(papers: List[Dict], count: int = 3) -> List[Dict]:
    has_pdf = filter_papers_with_pdfs(papers)
    ranked = rank_papers(has_pdf)
    selected = ranked[:count]
    print(f"Selected top {len(selected)} papers.")
    for i, p in enumerate(selected, start=1):
        title = p.get("title", "")[:80]
        print(f" {i}. {title} | Year: {p.get('year')} | Citations: {p.get('citationCount')}")
    logging.info(f"Selected {len(selected)} papers for download.")
    return selected

# -----------------------
# 3. DOWNLOAD with verification
# -----------------------
def is_pdf_content(response: requests.Response, first_bytes: bytes) -> bool:
    # Content-type check
    ctype = response.headers.get("content-type", "").lower()
    if "application/pdf" in ctype:
        return True
    # Magic bytes check
    if first_bytes.startswith(b"%PDF"):
        return True
    # Fallback: check for common PDF markers in first bytes
    if b"/Type /Catalog" in first_bytes or b"/Producer" in first_bytes:
        return True
    return False

def download_file_stream(url: str, dest_path: str, max_retries: int = 3, backoff_factor: float = 1.5) -> bool:
    headers = {"User-Agent": "Mozilla/5.0 (compatible; PaperDownloader/1.0)"}
    attempt = 0
    while attempt < max_retries:
        try:
            attempt += 1
            resp = requests.get(url, headers=headers, stream=True, timeout=30)
            if resp.status_code != 200:
                logging.warning(f"Download attempt {attempt} returned status {resp.status_code} for URL: {url}")
                time.sleep(backoff_factor * attempt)
                continue

            # read a small chunk to check magic bytes / content-type
            first_chunk = resp.raw.read(4096) or b""
            if not is_pdf_content(resp, first_chunk):
                # Not a PDF according to checks
                logging.warning(f"Downloaded content does not appear to be PDF (headers: {resp.headers.get('content-type')}) for {url}")
                # but sometimes redirectors serve PDF after streaming; attempt writing and verifying
            # Write file with progress bar if content-length known
            total_size = int(resp.headers.get("content-length") or 0)
            with open(dest_path, "wb") as f:
                # write the first chunk
                if first_chunk:
                    f.write(first_chunk)
                # Stream remaining content with progress
                if total_size:
                    remaining = total_size - len(first_chunk)
                    with tqdm(total=remaining, unit="B", unit_scale=True, desc=f"Downloading", leave=False) as pbar:
                        for chunk in resp.iter_content(chunk_size=8192):
                            if chunk:
                                f.write(chunk)
                                pbar.update(len(chunk))
                else:
                    # unknown size
                    for chunk in resp.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

            # Basic file size check
            if os.path.getsize(dest_path) < 2048:  # <2KB suspicious
                logging.warning(f"Downloaded file too small: {dest_path} (size={os.path.getsize(dest_path)})")
                # remove and retry
                os.remove(dest_path)
                time.sleep(backoff_factor * attempt)
                continue

            # Try opening with PyMuPDF
            try:
                with fitz.open(dest_path) as doc:
                    if len(doc) < 1:
                        logging.warning("PDF opened but has 0 pages.")
                        os.remove(dest_path)
                        time.sleep(backoff_factor * attempt)
                        continue
            except Exception as e:
                logging.warning(f"PyMuPDF failed to open file: {e}")
                # remove and retry
                if os.path.exists(dest_path):
                    os.remove(dest_path)
                time.sleep(backoff_factor * attempt)
                continue

            # Success
            return True

        except requests.exceptions.RequestException as re:
            logging.warning(f"Request error on attempt {attempt} for {url}: {re}")
            time.sleep(backoff_factor * attempt)
        except Exception as e:
            logging.exception(f"Unexpected error while downloading {url}: {e}")
            time.sleep(backoff_factor * attempt)

    return False

def download_selected_papers(selected_papers: List[Dict], output_dir: str = "downloads/pdfs") -> List[Dict]:
    downloaded = []
    for idx, paper in enumerate(selected_papers, start=1):
        title = paper.get("title", "paper")
        safe_title = safe_filename(title, max_len=80)
        # create filename with index + sha1 of title for uniqueness
        base_name = f"{idx}_{safe_title}"
        filename = os.path.join(output_dir, f"{base_name}.pdf")
        # if file exists, append timestamp
        if os.path.exists(filename):
            stamp = datetime.now().strftime("%Y%m%d%H%M%S")
            filename = os.path.join(output_dir, f"{base_name}_{stamp}.pdf")

        pdf_url = paper.get("pdf_url") or paper.get("url") or ""
        print(f"\n[{idx}/{len(selected_papers)}] Downloading: {title[:100]}")
        logging.info(f"Starting download for: {title} | URL: {pdf_url}")

        success = download_file_stream(pdf_url, filename, max_retries=4, backoff_factor=2.0)

        if not success:
            logging.error(f"Failed to download: {title} | URL: {pdf_url}")
            paper_meta = {
                "title": title,
                "paperId": paper.get("paperId"),
                "downloaded": False,
                "error": "download_failed",
                "pdf_url": pdf_url
            }
            downloaded.append(paper_meta)
            print("  ❌ Failed to download.")
            continue

        # compute metadata
        try:
            size_bytes = os.path.getsize(filename)
            sha256 = sha256_of_file(filename)
            with fitz.open(filename) as doc:
                pages = len(doc)
            pdf_info = {
                "local_path": filename,
                "size_bytes": size_bytes,
                "size_mb": round(size_bytes / (1024*1024), 3),
                "pages": pages,
                "sha256": sha256,
                "download_time": datetime.now().isoformat()
            }

            # build combined paper metadata (for dataset)
            paper_meta = {
                "title": paper.get("title"),
                "authors": paper.get("authors"),
                "year": paper.get("year"),
                "paperId": paper.get("paperId"),
                "citationCount": paper.get("citationCount"),
                "venue": paper.get("venue"),
                "pdf_url": pdf_url,
                "downloaded": True,
                "pdf_info": pdf_info,
                # placeholder fields for later modules:
                "abstract": paper.get("abstract"),
                "summary": None,        # to be filled by summarizer
                "keywords": None,       # optional extraction later
                "embeddings": None      # optional embeddings later
            }

            # Save per-paper metadata json
            meta_fname = safe_filename(paper.get("paperId") or paper.get("title")[:40])
            meta_path = os.path.join("downloads/metadata", f"{meta_fname}.json")
            with open(meta_path, "w", encoding="utf-8") as mf:
                json.dump(paper_meta, mf, indent=4, ensure_ascii=False)

            logging.info(f"Downloaded & verified: {filename} | pages={pages} | size={size_bytes}")
            print(f"  ✅ Downloaded. Pages: {pages} | Size: {paper_meta['pdf_info']['size_mb']} MB")
            downloaded.append(paper_meta)

        except Exception as e:
            logging.exception("Error processing downloaded file")
            print(f"  ⚠ Error processing file: {e}")
            # mark failure in metadata
            paper_meta = {
                "title": paper.get("title"),
                "paperId": paper.get("paperId"),
                "downloaded": False,
                "error": "postprocess_failed"
            }
            downloaded.append(paper_meta)
            # try to remove corrupted file
            try:
                if os.path.exists(filename):
                    os.remove(filename)
            except:
                pass

    return downloaded

# -----------------------
# 4. SAVE REPORT & BUILD DATASET (CSV + JSONL)
# -----------------------
def save_download_report_and_dataset(downloaded_papers: List[Dict], topic: str):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_path = f"data/reports/download_report_{timestamp}.json"

    report = {
        "topic": topic,
        "download_timestamp": datetime.now().isoformat(),
        "total_selected": len(downloaded_papers),
        "successful_downloads": sum(1 for p in downloaded_papers if p.get("downloaded")),
        "failed_downloads": sum(1 for p in downloaded_papers if not p.get("downloaded")),
        "papers": downloaded_papers
    }

    # Save full report
    with open(report_path, "w", encoding="utf-8") as rf:
        json.dump(report, rf, indent=4, ensure_ascii=False)
    logging.info(f"Saved download report: {report_path}")
    print(f"\nDownload report saved: {report_path}")

    # Build dataset CSV + JSONL (Option C)
    dataset_rows = []
    jsonl_path = f"data/dataset/papers_{timestamp}.jsonl"
    csv_path = f"data/dataset/papers_{timestamp}.csv"

    with open(jsonl_path, "w", encoding="utf-8") as jl:
        for p in downloaded_papers:
            row = {
                "title": p.get("title"),
                "authors": "; ".join(p.get("authors") or []),
                "year": p.get("year"),
                "paperId": p.get("paperId"),
                "citationCount": p.get("citationCount"),
                "venue": p.get("venue"),
                "pdf_url": p.get("pdf_url"),
                "local_path": (p.get("pdf_info") or {}).get("local_path"),
                "pages": (p.get("pdf_info") or {}).get("pages"),
                "size_mb": (p.get("pdf_info") or {}).get("size_mb"),
                "sha256": (p.get("pdf_info") or {}).get("sha256"),
                "downloaded": p.get("downloaded"),
                "download_time": (p.get("pdf_info") or {}).get("download_time"),
                "abstract": p.get("abstract"),
                "summary": p.get("summary"),
                "keywords": p.get("keywords")
            }
            dataset_rows.append(row)
            jl.write(json.dumps(row, ensure_ascii=False) + "\n")

    # Save CSV using pandas
    try:
        df = pd.DataFrame(dataset_rows)
        df.to_csv(csv_path, index=False, encoding="utf-8")
        logging.info(f"Saved dataset CSV: {csv_path}")
        print(f"Dataset saved: {csv_path}")
    except Exception as e:
        logging.exception("Failed to save CSV dataset")
        print(f"Failed to save CSV dataset: {e}")

    print(f"JSONL saved: {jsonl_path}")
    return report_path, csv_path, jsonl_path

# -----------------------
# 5. VERIFY DOWNLOADS FOLDER (Summary)
# -----------------------
def verify_downloads_folder(output_dir="downloads/pdfs"):
    """
    Summarize verification of files in downloads/pdfs
    """
    if not os.path.exists(output_dir):
        print(f"No downloads directory at {output_dir}")
        return

    files = [f for f in os.listdir(output_dir) if f.lower().endswith(".pdf")]
    total_size = 0
    valid_count = 0

    print("\nVerification summary for downloads:")
    for f in files:
        p = os.path.join(output_dir, f)
        size = os.path.getsize(p)
        total_size += size
        valid = False
        try:
            with fitz.open(p) as doc:
                if len(doc) > 0:
                    valid = True
                    valid_count += 1
            print(f" - {f} | {size/1024/1024:.2f} MB | pages: {len(doc)} | valid: {valid}")
        except Exception:
            print(f" - {f} | {size/1024/1024:.2f} MB | INVALID")
    print(f"\nTotal PDFs: {len(files)} | Valid: {valid_count} | Total size: {total_size/1024/1024:.2f} MB")
    return valid_count

# -----------------------
# 6. MAIN DOWNLOAD FLOW
# -----------------------
def main_download(filepath: str = None, download_count: int = 3):
    print("\n" + "="*80)
    print("MODULE 2: PAPER SELECTION & PDF DOWNLOAD")
    print("="*80)

    data = load_search_results(filepath)
    if not data:
        return None

    selected = select_top_papers(data.get("papers", []), count=download_count)
    if not selected:
        print("No suitable papers found for download.")
        return None

    downloaded = download_selected_papers(selected)
    report_path, csv_path, jsonl_path = save_download_report_and_dataset(downloaded, data.get("topic"))

    # Verification summary
    verify_downloads_folder("downloads/pdfs")

    print("\nModule 2 complete!")
    print(f" - Downloads: downloads/pdfs/")
    print(f" - Per-paper metadata: downloads/metadata/")
    print(f" - Report: {report_path}")
    print(f" - Dataset CSV: {csv_path}")
    print(f" - Dataset JSONL: {jsonl_path}")
    logging.info("Module 2 complete.")
    return {
        "report": report_path,
        "csv": csv_path,
        "jsonl": jsonl_path,
        "downloaded": downloaded
    }

# Run when module executed directly
if __name__ == "__main__":
    main_download(download_count=3)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 75.9 MB/s eta 0:00:00

MODULE 2: PAPER SELECTION & PDF DOWNLOAD
Loading most recent search results: yoga_search_results.json
Loaded 974 papers for topic 'yoga'
Selected top 3 papers.
 1. Interventions Shown to Aid Executive Function Development in Children 4 to 12 Ye | Year: 2011 | Citations: 2715
 2. Cognitive and emotional control of pain and its disruption in chronic pain | Year: 2013 | Citations: 1687
 3. Activities and Programs That Improve Children’s Executive Functions | Year: 2012 | Citations: 681

[1/3] Downloading: Interventions Shown to Aid Executive Function Development in Children 4 to 12 Years Old


  ✅ Downloaded. Pages: 14 | Size: 2.336 MB

[2/3] Downloading: Cognitive and emotional control of pain and its disruption in chronic pain


ERROR:root:Failed to download: Cognitive and emotional control of pain and its disruption in chronic pain | URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4465351


  ❌ Failed to download.

[3/3] Downloading: Activities and Programs That Improve Children’s Executive Functions


  ✅ Downloaded. Pages: 12 | Size: 0.242 MB

Download report saved: data/reports/download_report_20251212_102116.json
Dataset saved: data/dataset/papers_20251212_102116.csv
JSONL saved: data/dataset/papers_20251212_102116.jsonl

Verification summary for downloads:
 - 3_Activities and Programs That Improve Childrens Executive Functions.pdf | 0.24 MB | INVALID
 - 1_Interventions Shown to Aid Executive Function Development in Children 4 to 12 Ye.pdf | 2.34 MB | INVALID

Total PDFs: 2 | Valid: 2 | Total size: 2.58 MB

Module 2 complete!
 - Downloads: downloads/pdfs/
 - Per-paper metadata: downloads/metadata/
 - Report: data/reports/download_report_20251212_102116.json
 - Dataset CSV: data/dataset/papers_20251212_102116.csv
 - Dataset JSONL: data/dataset/papers_20251212_102116.jsonl
